In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
#import cv2
import torch
import sys
import nibabel as nib
from tqdm import tqdm

sys.path.append('/workspace/MRI-inpainting-project/data_scripts')
sys.path.append('/workspace/MRI-inpainting-project/I2SB/dataset')

from datasets import TrainPatchesDataset, HealthyMRIDataset, PathologicalMRIDataset
from mri_datasets import ReconMRIDataset
from visualization_utils import ImageSliceViewer3D

In [77]:
healthy_dataset = HealthyMRIDataset('/workspace/MRI-inpainting-project/data/healthy_mri_C_00/healthy_mri_t1',
                                    '/workspace/MRI-inpainting-project/data/healthy_mri_C_00/healthy_masks_t1/3dunet_masks_new/masks/split7/masks',
                                    '/workspace/MRI-inpainting-project/data/healthy_mri_C_00/healthy_masks_t1/3dunet_masks_new/patch_masks/split7')

recons_path = "/workspace/MRI-inpainting-project/I2SB/results/Split7T1/samples_nfe999/reconSplit7Unet.pt"

In [58]:
old_recons = torch.load("/workspace/MRI-inpainting-project/I2SB/results/Split0Train_v2/samples_nfe999/recon.pt")['arr']

In [59]:
new_recons = torch.load(recons_path)['arr']

In [69]:
new_filenames_order = healthy_dataset.filenames
old_zhores_filenames_order = ['sub-00067_t1_brain-final.nii.gz', 'sub-00028_t1_brain-final.nii.gz', 'sub-00148_t1_brain-final.nii.gz', 'sub-00163_t1_brain-final.nii.gz', 'sub-00152_t1_brain-final.nii.gz', 'sub-00093_t1_brain-final.nii.gz', 'sub-00046_t1_brain-final.nii.gz', 'sub-00052_t1_brain-final.nii.gz', 'sub-C10_t1_brain-final.nii.gz', 'sub-00088_t1_brain-final.nii.gz', 'sub-00049_t1_brain-final.nii.gz', 'sub-00150_t1_brain-final.nii.gz', 'sub-00117_t1_brain-final.nii.gz', 'sub-00007_t1_brain-final.nii.gz', 'sub-00165_t1_brain-final.nii.gz', 'sub-00069_t1_brain-final.nii.gz', 'sub-00056_t1_brain-final.nii.gz', 'sub-00113_t1_brain-final.nii.gz', 'sub-00054_t1_brain-final.nii.gz', 'sub-00143_t1_brain-final.nii.gz', 'sub-00070_t1_brain-final.nii.gz', 'sub-00013_t1_brain-final.nii.gz', 'sub-C1_t1_brain-final.nii.gz', 'sub-00119_t1_brain-final.nii.gz', 'sub-00127_t1_brain-final.nii.gz', 'sub-00075_t1_brain-final.nii.gz', 'sub-00025_t1_brain-final.nii.gz', 'sub-00002_t1_brain-final.nii.gz', 'sub-00057_t1_brain-final.nii.gz', 'sub-00096_t1_brain-final.nii.gz', 'sub-00168_t1_brain-final.nii.gz', 'sub-00036_t1_brain-final.nii.gz', 'sub-00169_t1_brain-final.nii.gz', 'sub-00153_t1_brain-final.nii.gz', 'sub-00114_t1_brain-final.nii.gz', 'sub-00017_t1_brain-final.nii.gz', 'sub-00030_t1_brain-final.nii.gz', 'sub-00051_t1_brain-final.nii.gz', 'sub-00155_t1_brain-final.nii.gz', 'sub-C6_t1_brain-final.nii.gz', 'sub-C2_t1_brain-final.nii.gz', 'sub-00157_t1_brain-final.nii.gz', 'sub-00102_t1_brain-final.nii.gz', 'sub-00154_t1_brain-final.nii.gz', 'sub-00135_t1_brain-final.nii.gz', 'sub-C11_t1_brain-final.nii.gz', 'sub-00035_t1_brain-final.nii.gz', 'sub-00110_t1_brain-final.nii.gz', 'sub-00147_t1_brain-final.nii.gz', 'sub-00104_t1_brain-final.nii.gz', 'sub-00129_t1_brain-final.nii.gz', 'sub-00111_t1_brain-final.nii.gz', 'sub-00031_t1_brain-final.nii.gz', 'sub-00170_t1_brain-final.nii.gz', 'sub-00039_t1_brain-final.nii.gz', 'sub-00042_t1_brain-final.nii.gz', 'sub-00085_t1_brain-final.nii.gz', 'sub-00079_t1_brain-final.nii.gz', 'sub-00012_t1_brain-final.nii.gz', 'sub-C8_t1_brain-final.nii.gz', 'sub-00118_t1_brain-final.nii.gz', 'sub-00029_t1_brain-final.nii.gz', 'sub-00084_t1_brain-final.nii.gz', 'sub-00011_t1_brain-final.nii.gz', 'sub-00019_t1_brain-final.nii.gz', 'sub-00162_t1_brain-final.nii.gz', 
                              'sub-00037_t1_brain-final.nii.gz', 'sub-00008_t1_brain-final.nii.gz', 'sub-00124_t1_brain-final.nii.gz', 'sub-00022_t1_brain-final.nii.gz', 'sub-00021_t1_brain-final.nii.gz', 'sub-00137_t1_brain-final.nii.gz', 'sub-00149_t1_brain-final.nii.gz', 'sub-00106_t1_brain-final.nii.gz', 'sub-00164_t1_brain-final.nii.gz', 'sub-00005_t1_brain-final.nii.gz', 'sub-00041_t1_brain-final.nii.gz']

In [60]:
map_ids = []
for i in tqdm(range(len(new_recons))):
    map_ids.append(np.argmin((np.abs(old_recons - new_recons[i])).sum(axis=(1, 2, 3))).item())

100%|██████████| 1232/1232 [09:28<00:00,  2.17it/s]


In [73]:
recons_right_order = [0] * len(new_recons)
for i in range(len(new_recons)):
    recons_right_order[new_filenames_order.index(old_zhores_filenames_order[i // 16]) * 16 + (i % 16)] = new_recons[i]

recons_right_order = torch.stack(recons_right_order)

In [74]:
torch.save({'arr': recons_right_order, 'label_arr': torch.load(recons_path)['label_arr']}, "/workspace/MRI-inpainting-project/I2SB/results/Split3T1/samples_nfe999/recon_right_order.pt")

In [78]:
recon_mri_dataset = ReconMRIDataset(recons_path, healthy_dataset, flair=False)

In [79]:
sample = recon_mri_dataset[11]
mri_recon, mask = sample['mri_recon'], sample['mask']

In [80]:
ImageSliceViewer3D(mri_recon, mask, cmap='gray', mask_intensity=0.2)

interactive(children=(RadioButtons(description='Slice plane selection:', options=('x-y', 'y-z', 'z-x'), style=…

In [81]:
for sample in tqdm(recon_mri_dataset):
    mri_recon, mask, filename = sample['mri_recon'], sample['mask'], sample['filename']
    
    mri_recon = nib.Nifti1Image(mri_recon, affine=np.eye(4))
    mask = nib.Nifti1Image(mask, affine=np.eye(4))

    nib.save(mri_recon, "/workspace/Features/Features/generated_mri/i2sb/3dunet_masks_new/split7/mri/" + filename)
    #nib.save(mask,      "/workspace/Features/Features/generated_mri/i2sb/sphere_masks/masks/" + filename)

100%|██████████| 77/77 [01:40<00:00,  1.31s/it]


In [82]:
len(os.listdir("/workspace/Features/Features/generated_mri/i2sb/3dunet_masks_new/split7/mri"))

77